In [213]:
import pandas as pd
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
import os
import cv2
import matplotlib.pyplot as plt
import random
from numpy import save

In [244]:
df2= df[df['labels']== "['D']"]

In [265]:
df3= df[(df['labels']!= "['D']") & (df['D']== 0)]

In [268]:
Diab=list(df2.filename)

In [269]:
len(Diab)

1608

In [270]:
NoDiab=list(df3.filename)

In [271]:
len(NoDiab)

4269

In [216]:
df=pd.read_csv('full_df.csv')

In [217]:
def has_diabetes(x):
    if 'proliferative' in x:
        return 1
    elif 'nonproliferative' in x:
        return 1
    elif 'non proliferative' in x:
        return 1
    elif 'diabetic'in x:
        return 1
    else:
        return 0

In [218]:
df["left_diabetes"] = df["Left-Diagnostic Keywords"].apply(lambda x: has_diabetes(x))
df["right_diabetes"] = df["Right-Diagnostic Keywords"].apply(lambda x: has_diabetes(x))

In [219]:
left_diabetes = df.loc[(df.left_diabetes == 1)]["Left-Fundus"].values
right_diabetes = df.loc[(df.right_diabetes == 1)]["Right-Fundus"].values

In [220]:
diabetes = np.concatenate((left_diabetes,right_diabetes),axis=0)

In [221]:
len(diabetes)

3411

In [222]:
Diabetes= set(diabetes)

In [223]:
len(Diabetes)

1764

In [228]:
left_normal = df.loc[(df.D ==0)]["Left-Fundus"].sample(960,random_state=42).values
right_normal = df.loc[(df.D==0)]["Right-Fundus"].sample(960,random_state=42).values

In [229]:
Nodiab = np.concatenate((left_normal,right_normal),axis=0)

In [230]:
NOdiab=set(Nodiab)

In [231]:
len(NOdiab)

1726

In [253]:
def create_dataset(image_category,label):
    dataset_dir = "Datos/preprocessed_2"
    image_size=250
    labels = []
    dataset = []
    for img in image_category:
        image_path = os.path.join(dataset_dir,img)
        try:
            img = cv2.imread(image_path,0)
            img = cv2.resize(img, (250,250))
            img = img.astype(np.float32)/255
            img = img[..., np.newaxis]
            
        except:
            continue
        
        dataset.append([np.array(img),np.array(label)])
    random.shuffle(dataset)
    return dataset

In [254]:
def augment(category,label,mult):
    x=[]
    for img in category:
        try:
            img = cv2.imread(f"Datos/preprocessed_2/{img}",0)
            img = cv2.resize(img, (250,250))
            img = img.astype(np.float32)/255
            img = img[..., np.newaxis]
        except:
            continue
        train_datagen = ImageDataGenerator(
                    rotation_range=30,
                    zoom_range=0.1,
                    vertical_flip=True,
                    brightness_range=(0.2, 1.5),
                    height_shift_range=0.1,
                    )
        img = img.reshape((1,) + img.shape)
        
        it = train_datagen.flow(x=img, batch_size=1)
        for i in range(mult):
            batch = it.next()
            aug_img = batch[0]
            aug_img=aug_img.reshape(-1)
            x.append([np.array(img),np.array(label)])
            
       
    for i in range(len(x)):
        x[i][0]=x[i][0].reshape(x[i][0].shape[1:])
    return x

In [272]:
NoDiabetes= create_dataset(NoDiab,0)

In [273]:
Dia=create_dataset(Diab,1)

In [274]:
DiabAu=augment(Diab,1,2)

In [275]:
DiabetesFull=Dia+NoDiabetes+DiabAu

In [276]:
len(DiabetesFull)

9093

In [260]:
image_size=250

In [277]:
x = np.array([i[0]for i in DiabetesFull])
y = np.array([i[1] for i in DiabetesFull])

In [278]:
x.shape

(9093, 250, 250, 1)

In [279]:
save('x',x)

In [280]:
save('y',y)